In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 
from requests_html import AsyncHTMLSession
import httpx
import asyncio
import concurrent.futures
import time
import math

In [2]:
#Gathering advanced stats table
table_df = pd.DataFrame(columns = ["Year", "Team", "Age", "GP", "W", "L", "W/L%"])
years = [2012, 2013, 2014, 2015, 
         2016, 2017, 2018, 2019, 2020, 2021, 2022] 

for y in years:
        
        year = y
        df = pd.read_html(f'https://www.basketball-reference.com/leagues/NBA_{year}.html', match = "Advanced Stats")
        temp_df = pd.DataFrame(columns = ["Year", "Team", "Age","GP", "W", "L", "W/L%", "Avg Exp", "Coach Exp", "Coach W/L%", "Avg PER", "Avg WS"])
        temp_df['Team'] = df[0]['Unnamed: 1_level_0']['Team']
        temp_df['Age'] = df[0]['Unnamed: 2_level_0']['Age']
        temp_df['W'] = df[0]['Unnamed: 3_level_0']['W']
        temp_df['L'] = df[0]['Unnamed: 4_level_0']['L']
        temp_df['GP'] = temp_df['W'] + temp_df['L']
        temp_df['W/L%'] = temp_df['W'] / (temp_df['W'] + temp_df['L'])
        temp_df['Year'] = y
        table_df = table_df.append(temp_df)
            
table_df.head()

,Year,Team,Age,GP,W,L,W/L%,Avg Exp,Coach Exp,Coach W/L%,Avg PER,Avg WS
0,2012,Chicago Bulls*,27.1,66.0,50.0,16.0,0.757576,NaN,NaN,NaN,NaN,NaN
1,2012,San Antonio Spurs*,27.5,66.0,50.0,16.0,0.757576,NaN,NaN,NaN,NaN,NaN
2,2012,Oklahoma City Thunder*,25.1,66.0,47.0,19.0,0.712121,NaN,NaN,NaN,NaN,NaN
3,2012,Miami Heat*,28.3,66.0,46.0,20.0,0.696970,NaN,NaN,NaN,NaN,NaN
4,2012,Philadelphia 76ers*,24.9,66.0,35.0,31.0,0.530303,NaN,NaN,NaN,NaN,NaN


In [3]:
#Removing asteriks in team names
table_df["Team"] = table_df["Team"].str.replace('*', '')

/var/folders/mh/bzcftdl55cj6hrvx6vqfym1m0000gq/T/ipykernel_64610/838356802.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  table_df["Team"] = table_df["Team"].str.replace('*', '')


In [4]:
#Gathering average experience for each team
team_urls = []
for y in years:
        year = y
        url = f'https://www.basketball-reference.com/leagues/NBA_{year}.html'
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        table = soup.find('table', {'id':'advanced-team'})
        rows = table.find_all('tr')
        
        for row in rows:
            try:
                link = row.select('a')[0]['href']
                team_urls.append(link)
            except:
                continue
                
def experience(link):
            url = "https://www.basketball-reference.com" + link
            df = pd.read_html(url, match = "Roster")
            temp_df = pd.DataFrame(columns = ["Exp"])
            temp_df['Exp'] = df[0]['Exp']
            temp_df['Exp'] = temp_df['Exp'].replace('R',0)
            temp_df['Exp'] = temp_df['Exp'].astype(float)
            
            r = requests.get(url)
            soup = BeautifulSoup(r.text, 'html.parser')
            year = soup.find('h1').find_all('span')
            year = year[0].text
            year = year[ : 4]
            year = int(year)
            year = year + 1
            team = soup.find('h1').find_all('span')
            team = team[1].text
            
            table_df.loc[((table_df['Year'] == year) & (table_df['Team'] == team)), 'Avg Exp'] = temp_df['Exp'].mean()

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(experience, team_urls)

In [5]:
table_df.to_csv("table2.csv")

In [6]:
#Gathering average experience for each team
team_urls = []
for y in years:
        year = y
        url = f'https://www.basketball-reference.com/leagues/NBA_{year}.html'
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        table = soup.find('table', {'id':'advanced-team'})
        rows = table.find_all('tr')
        
        for row in rows:
            try:
                link = row.select('a')[0]['href']
                team_urls.append(link)
            except:
                continue
                
def experience(link):
            url = "https://www.basketball-reference.com" + link
            df = pd.read_html(url, match = "Roster")
            temp_df = pd.DataFrame(columns = ["Exp"])
            temp_df['Exp'] = df[0]['Exp']
            temp_df['Exp'] = temp_df['Exp'].replace('R',0)
            temp_df['Exp'] = temp_df['Exp'].astype(float)
            
            r = requests.get(url)
            soup = BeautifulSoup(r.text, 'html.parser')
            year = soup.find('h1').find_all('span')
            year = year[0].text
            year = year[ : 4]
            year = int(year)
            year = year + 1
            team = soup.find('h1').find_all('span')
            team = team[1].text
            
            table_df.loc[((table_df['Year'] == year) & (table_df['Team'] == team)), 'Avg Exp'] = temp_df['Exp'].mean()

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(experience, team_urls)

In [7]:
table_df.to_csv("table2.csv")

In [8]:
#Gathering coaches' W/L% & Experience       
def coaches(link):
            url = "https://www.basketball-reference.com" + link
            r = requests.get(url)
            soup = BeautifulSoup(r.text, 'html.parser')
            team = soup.find('h1').find_all('span')
            team = team[1].text
            year = soup.find('h1').find_all('span')
            year = year[0].text
            year = year[ : 4]
            year = int(year)
            year = year + 1
            info = soup.find('div', {'data-template': 'Partials/Teams/Summary'})
            coach = info.find('p').findNext('p').select('a')[0]['href']
            coach_url = "https://www.basketball-reference.com" + coach
            
            df = pd.read_html(coach_url, match = "Coaching Record")
            temp_df = pd.DataFrame(columns = ["Season", "G", "W/L%"])
            temp_df["Season"] = df[0]['Unnamed: 0_level_0']['Season']
            temp_df["G"] = df[0]['Regular Season']['G']
            temp_df["W/L%"] = df[0]['Regular Season']['W/L%']
            temp_df['G'] = temp_df['G'].astype(str)
            temp_df = temp_df[temp_df["G"].str.contains("Assistant")==False]
            temp_df = temp_df[temp_df["G"].str.contains("Associate")==False]
            temp_df = temp_df[temp_df["G"].str.contains("Player Development")==False]
            temp_df = temp_df[temp_df["Season"] != "Career"]
            temp_df = temp_df[temp_df["Season"].str.contains("seasons")==False]
            temp_df = temp_df[temp_df["Season"].str.contains("season")==False]
            temp_df["Season"] = temp_df["Season"].str[:-3]
            temp_df['Season'] = temp_df['Season'].astype(int)
            temp_df["Season"] = temp_df["Season"].apply(lambda x: x +1)
            temp_df = temp_df.loc[temp_df["Season"] < year]
            coach_experience = len(temp_df)
            temp_df['W/L%'] = temp_df['W/L%'].astype(float)
            coach_wl = temp_df['W/L%'].mean()
            table_df.loc[((table_df['Year'] == year) & (table_df['Team'] == team)), 'Coach Exp'] = coach_experience
            table_df.loc[((table_df['Year'] == year) & (table_df['Team'] == team)), 'Coach W/L%'] = coach_wl

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(coaches, team_urls)

In [9]:
table_df.to_csv("table2.csv")

In [10]:
#Gathering PER & WS
def advanced(link):
    num_players = 0.0
    total_avg_PER = 0.0
    total_avg_WS = 0.0
    player_urls = []
    url = "https://www.basketball-reference.com" + link
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    year = soup.find('h1').find_all('span')
    year = year[0].text
    year = year[ : 4]
    year = int(year)
    year = year + 1
    team = soup.find('h1').find_all('span')
    team = team[1].text
    
    table = soup.find('table', {'id':'roster'})
    rows = table.find_all('tr')
        
    for row in rows:
        try:
            if ((row.find('td', {'data-stat': 'years_experience'}).text != 'R') and (row.find('td', {'data-stat': 'player'}).find('a').text != 'KZ Okpala')): 
                link = row.select('a')[0]['href']
                player_urls.append(link)
        except:
            continue
            
    for link in player_urls:
        url = "https://www.basketball-reference.com" + link
        df = pd.read_html(url, match = "Advanced")
                         
        temp_df = pd.DataFrame(columns = ["Season", "PER", "WS"])
        temp_df["Season"] = df[0]['Season']
        temp_df["PER"] = df[0]['PER']
        temp_df["WS"] = df[0]['WS']
        temp_df = temp_df[temp_df["Season"].str.contains("seasons")==False]
        temp_df = temp_df[temp_df["Season"].str.contains("season")==False]
        temp_df = temp_df[temp_df["Season"] != "Career"]
        temp_df["Season"] = temp_df["Season"].str[:-3]
        temp_df['Season'] = temp_df['Season'].astype(int)
        temp_df["Season"] = temp_df["Season"].apply(lambda x: x + 1)
        temp_df.drop_duplicates(subset = "Season", inplace = True)
        temp_df = temp_df.loc[temp_df["Season"] < year]
                         
        avg_PER = temp_df['PER'].mean()
        avg_WS = temp_df['WS'].mean()
        if (math.isnan(avg_PER) != True and math.isnan(avg_WS) != True):
            total_avg_PER = total_avg_PER + avg_PER
            total_avg_WS = total_avg_WS + avg_WS
            num_players = num_players + 1
    
    final_avg_PER = total_avg_PER / num_players
    final_avg_WS = total_avg_WS / num_players
    table_df.loc[((table_df['Year'] == year) & (table_df['Team'] == team)), 'Avg PER'] = final_avg_PER
    table_df.loc[((table_df['Year'] == year) & (table_df['Team'] == team)), 'Avg WS'] = final_avg_WS
                      
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(advanced, team_urls)

In [11]:
table_df.to_csv("table2.csv")

In [12]:
#Keeping team names consistent and removing unneccesary League Average row
#table_df["Team"] = table_df["Team"].str.replace('Seattle SuperSonics', 'Oklahoma City Thunder')
table_df["Team"] = table_df["Team"].str.replace('New Jersey Nets', 'Brooklyn Nets')
#table_df["Team"] = table_df["Team"].str.replace('Washington Bullets', 'Washington Wizards')
#table_df["Team"] = table_df["Team"].str.replace('Vancouver Grizzlies', 'Memphis Grizzlies')

#table_df.loc[((table_df['Year'] < 2003) & (table_df['Team'] == 'Charlotte Hornets')), 'Team'] = 'New Orleans Pelicans'
table_df["Team"] = table_df["Team"].str.replace('New Orleans Hornets', 'New Orleans Pelicans')
#table_df["Team"] = table_df["Team"].str.replace('New Orleans/Oklahoma City Hornets', 'New Orleans Pelicans')
table_df["Team"] = table_df["Team"].str.replace('Charlotte Hornets', 'Charlotte Bobcats')

table_df = table_df[table_df['Team'] != 'League Average']

In [13]:
table_df.to_csv("table2.csv")

In [14]:
#Sorting table by team and year
table_df = table_df.sort_values(by=['Team','Year'])

In [15]:
table_df.to_csv("table2.csv")

In [16]:
#Rounding W/L and Avg Exp colummns and creating Last Year W/L column
table_df['W'] = table_df['W'].astype(int)
table_df['W/L%'] = table_df['W/L%'].round(3)
table_df["Coach W/L%"].fillna(0, inplace = True)
table_df['Coach W/L%'] = table_df['Coach W/L%'].round(3)
table_df['Avg PER'] = table_df['Avg PER'].astype(float)
table_df['Avg WS'] = table_df['Avg WS'].astype(float)
table_df['Avg PER'] = table_df['Avg PER'].round(3)
table_df['Avg WS'] = table_df['Avg WS'].round(3)
table_df['Avg Exp'] = table_df['Avg Exp'].astype(float)
table_df['Avg Exp'] = table_df['Avg Exp'].round(3)
series_shifted = table_df['W/L%'].shift()
table_df['Last Year W/L%'] = series_shifted
table_df.to_csv('table2.csv')

In [17]:
table_df = table_df[table_df['Year'] != 2012]
table_df.to_csv('table2.csv')